In [1]:
from pycocotools.coco import COCO
import numpy as np
import os
import torch
from PIL import ExifTags, Image, ImageOps, ImageDraw, ImageFont
import time

class load_coco():
    def __init__(self, root_dir, set_name='train2017'):
        self.root_dir = root_dir
        self.set_name = set_name
        self.coco = COCO(os.path.join(self.root_dir, 'annotations', 'instances_' + self.set_name + '.json'))
        
        whole_image_ids = self.coco.getImgIds()  # remove the images which has no annot
        self.image_ids = []
        for idx in whole_image_ids:
            annotations_ids = self.coco.getAnnIds(imgIds=idx, iscrowd=False)
            if len(annotations_ids) != 0:
                self.image_ids.append(idx)
        
    def load_image(self, image_index):
            image_info = self.coco.loadImgs(self.image_ids[image_index])[0]
            path = os.path.join(self.root_dir, self.set_name, image_info['file_name'])
            image = Image.open(path).convert('RGB')
            return np.array(image)
        
    def load_annotations(self, image_index):
        # get ground truth annotations
        annotations_ids = self.coco.getAnnIds(imgIds=self.image_ids[image_index], iscrowd=False)
        annotations = np.zeros((0, 4))

        # some images appear to miss annotations (like image with id 257034)
        if len(annotations_ids) == 0:
            return annotations

        # parse annotations
        coco_annotations = self.coco.loadAnns(annotations_ids)
        for idx, a in enumerate(coco_annotations):

            # some annotations have basically no width / height, skip them
            if a['bbox'][2] < 1 or a['bbox'][3] < 1:
                continue

            annotation = np.zeros((1, 4))
            annotation[0, :] = a['bbox']
            annotations = np.append(annotations, annotation, axis=0)

        # transform from [x, y, w, h] to [x1, y1, x2, y2]
        annotations[:, 2] = annotations[:, 0] + annotations[:, 2]
        annotations[:, 3] = annotations[:, 1] + annotations[:, 3]

        return annotations
    
    def get_dataset(self):
        
        print('We found {} files... Read images/labels...'.format(len(self.image_ids)))
        cnt = 0
        since = time.time()
        img_all, label_all = [], []
        for idx in range(5000):
            if cnt % 10000 == 0 and cnt != 0: 
                time_elapsed = time.time() - since
                print('\t Current idx {}... complete in {:.0f}m {:.0f}s'.format(cnt, time_elapsed // 60, time_elapsed % 60))
            img_all.append(self.load_image(idx))
            label_all.append(self.load_annotations(idx))
            cnt += 1
            
        return img_all, label_all

In [2]:
coco_loader = load_coco(root_dir='/nasdata2/khj/objectdetection/segmentation/coco', set_name='train2017')

loading annotations into memory...
Done (t=16.60s)
creating index...
index created!


In [3]:
img_all, label_all = coco_loader.get_dataset()

We found 117266 files... Read images/labels...


In [4]:
print(len(img_all))
print(np.shape(img_all[0]))

print(label_all[0])

5000
(360, 640, 3)
[[359.17 146.17 471.62 359.74]
 [339.88  22.16 493.76 322.89]
 [471.64 172.82 507.56 220.92]
 [486.01 183.31 516.64 218.29]]


In [4]:
len(coco_loader.image_ids)

117266

In [5]:
coco_loader.image_ids[0]

391895

In [11]:
from tensorized_GNN import BatchedGNNlayer
gnns = torch.nn.ModuleList([BatchedGNNlayer(96,96)])
gnns.append(BatchedGNNlayer(96,96))
gnns.append(BatchedGNNlayer(96,96))

ModuleList(
  (0): BatchedGNNlayer(
    (W): Linear(in_features=96, out_features=96, bias=True)
  )
  (1): BatchedGNNlayer(
    (W): Linear(in_features=96, out_features=96, bias=True)
  )
  (2): BatchedGNNlayer(
    (W): Linear(in_features=96, out_features=96, bias=True)
  )
)

In [15]:
import torchvision
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
# (optional) load the pre-trained weights

pretrained_dict = torch.load('./output_h2d_gnn_coco/ckpt_epoch8_lr0.001_h512_K64_L3_b1_nl2_t0.001_samp0.pth',map_location=torch.device('cpu')) 
#model.backbone.load_state_dict(pretrained_dict)

In [19]:
model.backbone.state_dict().keys()

odict_keys(['body.conv1.weight', 'body.bn1.weight', 'body.bn1.bias', 'body.bn1.running_mean', 'body.bn1.running_var', 'body.layer1.0.conv1.weight', 'body.layer1.0.bn1.weight', 'body.layer1.0.bn1.bias', 'body.layer1.0.bn1.running_mean', 'body.layer1.0.bn1.running_var', 'body.layer1.0.conv2.weight', 'body.layer1.0.bn2.weight', 'body.layer1.0.bn2.bias', 'body.layer1.0.bn2.running_mean', 'body.layer1.0.bn2.running_var', 'body.layer1.0.conv3.weight', 'body.layer1.0.bn3.weight', 'body.layer1.0.bn3.bias', 'body.layer1.0.bn3.running_mean', 'body.layer1.0.bn3.running_var', 'body.layer1.0.downsample.0.weight', 'body.layer1.0.downsample.1.weight', 'body.layer1.0.downsample.1.bias', 'body.layer1.0.downsample.1.running_mean', 'body.layer1.0.downsample.1.running_var', 'body.layer1.1.conv1.weight', 'body.layer1.1.bn1.weight', 'body.layer1.1.bn1.bias', 'body.layer1.1.bn1.running_mean', 'body.layer1.1.bn1.running_var', 'body.layer1.1.conv2.weight', 'body.layer1.1.bn2.weight', 'body.layer1.1.bn2.bias', 

In [21]:
pretrained_dict.keys()

odict_keys(['module.body.conv1.weight', 'module.body.bn1.weight', 'module.body.bn1.bias', 'module.body.bn1.running_mean', 'module.body.bn1.running_var', 'module.body.layer1.0.conv1.weight', 'module.body.layer1.0.bn1.weight', 'module.body.layer1.0.bn1.bias', 'module.body.layer1.0.bn1.running_mean', 'module.body.layer1.0.bn1.running_var', 'module.body.layer1.0.conv2.weight', 'module.body.layer1.0.bn2.weight', 'module.body.layer1.0.bn2.bias', 'module.body.layer1.0.bn2.running_mean', 'module.body.layer1.0.bn2.running_var', 'module.body.layer1.0.conv3.weight', 'module.body.layer1.0.bn3.weight', 'module.body.layer1.0.bn3.bias', 'module.body.layer1.0.bn3.running_mean', 'module.body.layer1.0.bn3.running_var', 'module.body.layer1.0.downsample.0.weight', 'module.body.layer1.0.downsample.1.weight', 'module.body.layer1.0.downsample.1.bias', 'module.body.layer1.0.downsample.1.running_mean', 'module.body.layer1.0.downsample.1.running_var', 'module.body.layer1.1.conv1.weight', 'module.body.layer1.1.b

In [23]:
#mod_dicts = [k.replace('module.',''):v for k,v in pretrained_dict]
mod_dicts = dict((key.replace('module.',''), value) for (key, value) in pretrained_dict.items())

In [25]:
mod_dicts.keys()

dict_keys(['body.conv1.weight', 'body.bn1.weight', 'body.bn1.bias', 'body.bn1.running_mean', 'body.bn1.running_var', 'body.layer1.0.conv1.weight', 'body.layer1.0.bn1.weight', 'body.layer1.0.bn1.bias', 'body.layer1.0.bn1.running_mean', 'body.layer1.0.bn1.running_var', 'body.layer1.0.conv2.weight', 'body.layer1.0.bn2.weight', 'body.layer1.0.bn2.bias', 'body.layer1.0.bn2.running_mean', 'body.layer1.0.bn2.running_var', 'body.layer1.0.conv3.weight', 'body.layer1.0.bn3.weight', 'body.layer1.0.bn3.bias', 'body.layer1.0.bn3.running_mean', 'body.layer1.0.bn3.running_var', 'body.layer1.0.downsample.0.weight', 'body.layer1.0.downsample.1.weight', 'body.layer1.0.downsample.1.bias', 'body.layer1.0.downsample.1.running_mean', 'body.layer1.0.downsample.1.running_var', 'body.layer1.1.conv1.weight', 'body.layer1.1.bn1.weight', 'body.layer1.1.bn1.bias', 'body.layer1.1.bn1.running_mean', 'body.layer1.1.bn1.running_var', 'body.layer1.1.conv2.weight', 'body.layer1.1.bn2.weight', 'body.layer1.1.bn2.bias', '